In [2]:
import sqlite3
import pandas as pd
import plotly.express as px

conn_mongoose = sqlite3.connect('mongoose_smells.db')
conn_mongodb = sqlite3.connect('mongoDB_smells.db')

In [3]:
schemas = pd.read_sql_query('select rowid, * from schema', conn_mongoose)
smells_mongoose = pd.read_sql_query('select * from smell', conn_mongoose)
smells_mongodb = pd.read_sql_query('select * from smell', conn_mongodb)
smells = pd.concat([smells_mongoose, smells_mongodb], ignore_index=True).drop_duplicates()
dbaccesses_mongoose = pd.read_sql_query('select * from dbaccess', conn_mongoose)
dbaccesses_mongodb = pd.read_sql_query('select * from dbaccess', conn_mongodb)
dbaccesses = pd.concat([dbaccesses_mongoose, dbaccesses_mongodb], ignore_index=True).drop_duplicates()

In [4]:
mongodb_projects = set(dbaccesses_mongodb['project_name'].values)
mongoose_projects = set(dbaccesses_mongoose['project_name'].values)

In [5]:
stars_mongoDB = pd.read_csv('stars_mongoDB.csv')[['repo', 'stars']].set_index('repo')
stars_mongoose = pd.read_csv('stars_mongoose.csv')[['repo', 'stars']].set_index('repo')
stars = pd.concat([stars_mongoDB, stars_mongoose])
grouped = stars.groupby(level=0)
stars = grouped.last()

In [6]:
locs_mongoDB = pd.read_csv('locs_mongoDB.csv')[['repo', 'js']].set_index('repo')
locs_mongoose = pd.read_csv('locs_mongoose.csv')[['repo', 'js']].set_index('repo')
locs = pd.concat([locs_mongoDB, locs_mongoose])
grouped = locs.groupby(level=0)
locs = grouped.last()

In [7]:
conn_commits_mongoose = sqlite3.connect('last_commit_mongoose.db')
conn_commits_mongoDB = sqlite3.connect('last_commit_mongoDB.db')
commits_mongoose = pd.read_sql_query('select * from correspondance', conn_commits_mongoose).set_index('project_name')
commits_mongoDB = pd.read_sql_query('select * from correspondance', conn_commits_mongoDB).set_index('project_name')
commits = pd.concat([commits_mongoose, commits_mongoDB])
grouped = commits.groupby(level=0)
commits = grouped.last()
commits['last_commit_hash'] = commits['last_commit_hash'].str.decode('utf-8').str.strip()
commits

,last_commit_hash
project_name,
01org/intel-iot-services-orchestration-layer,d1e2d3737375fdfe9a0b18e2a91da22050985a5c
0xTracker/0x-event-extractor,6b30a6453a857d7ca1661393a922005859c5ee8d
17koa/koa-generator-examples,59de28e6e46b11db8a1face1adc0cde41b83d248
18F/checklistomania,98657747c5460c51debb6e396e4ade254e63df53
1956-studio/wechat-ship,70a6ce289836c4d0878185a80e35b7e0758b11fa
...,...
zishon89us/node-cheat,e73252e81aed263c9639a94b6f97506e8aa036a2
zkweb-framework/ZKWeb,6ddd2ef8bbb448b3609f9eaaa953cf6be2b45acd
zrecore/zrecore-js,c03e1f8ad0068b02e7df9238d6d14cd8652e10df


In [8]:
locs

,js
repo,
01org/intel-iot-services-orchestration-layer,24772.0
0xTracker/0x-event-extractor,1344.0
17koa/koa-generator-examples,553.0
18F/checklistomania,1660.0
1956-studio/wechat-ship,2609.0
...,...
zishon89us/node-cheat,3233.0
zkweb-framework/ZKWeb,310.0
zrecore/zrecore-js,4227.0


In [9]:
schemas = pd.read_sql_query('select project_name, count(*) from schema group by project_name', conn_mongoose).set_index('project_name')
schemas

,count(*)
project_name,
0xTracker/0x-event-extractor,2
17koa/koa-generator-examples,1
1956-studio/wechat-ship,11
36node/sketch,1
7anshuai/abba,3
...,...
yunnysunny/nodebook-sample,4
yurykorzun/notinphilly,15
zackster/CompassionPit--Node-,11


In [11]:
db_accesses_mongoose = pd.read_sql_query('select project_name, count(*) from dbaccess group by project_name', conn_mongoose)
db_accesses_mongoDB = pd.read_sql_query('select project_name, count(*) from dbaccess group by project_name', conn_mongodb)
db_accesses_mongoose = db_accesses_mongoose.set_index('project_name')
db_accesses_mongoDB = db_accesses_mongoDB.set_index('project_name')
db_accesses = pd.concat([db_accesses_mongoose, db_accesses_mongoDB])
grouped = db_accesses.groupby(level=0)
db_accesses = grouped.last()

In [13]:
locs_db_access = locs.loc[locs.index.isin(db_accesses.index)]
locs_db_access

,js
repo,
0xTracker/0x-event-extractor,1344.0
17koa/koa-generator-examples,553.0
18F/checklistomania,1660.0
1956-studio/wechat-ship,2609.0
36node/sketch,8099.0
...,...
zaidka/genieacs,30.0
zipfworks/ember-prerender,1255.0
zishon89us/node-cheat,3233.0


In [14]:
stars_db_access = stars.loc[stars.index.isin(db_accesses.index)]
stars_db_access

,stars
repo,
0xTracker/0x-event-extractor,31
17koa/koa-generator-examples,485
18F/checklistomania,362
1956-studio/wechat-ship,13
36node/sketch,17
...,...
zaidka/genieacs,514
zipfworks/ember-prerender,141
zishon89us/node-cheat,413


In [15]:
stars_db_access['js'] = locs_db_access['js']
stars_db_access['dbaccesses'] = db_accesses['count(*)']
stars_db_access

/tmp/ipykernel_64743/4254591006.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stars_db_access['js'] = locs_db_access['js']
/tmp/ipykernel_64743/4254591006.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stars_db_access['dbaccesses'] = db_accesses['count(*)']


,stars,js,dbaccesses
repo,,,
0xTracker/0x-event-extractor,31,1344.0,7
17koa/koa-generator-examples,485,553.0,6
18F/checklistomania,362,1660.0,27
1956-studio/wechat-ship,13,2609.0,49
36node/sketch,17,8099.0,4
...,...,...,...
zaidka/genieacs,514,30.0,188
zipfworks/ember-prerender,141,1255.0,3
zishon89us/node-cheat,413,3233.0,19


In [16]:
stars_db_access.rename({'js': 'loc'}, axis='columns').to_csv('projects_stats.csv')

In [17]:
projects_stats = stars_db_access.rename({'js': 'loc'}, axis='columns')

In [87]:
smells.to_csv('smells.csv')

In [19]:
smells_occ = smells.groupby('project_name').count()['file_path']

In [20]:
smells_occ

project_name
EasyERP/EasyERP_open_source               49
Lambda-School-Labs/CS10-restaurant-pos     1
Requarks/wiki                              1
XimeraProject/server                       1
abcdabcd987/p2dv.in                        1
codefordenver/Comrad                       3
impronunciable/hackdash                    1
voluntarily/vly2                           1
Name: file_path, dtype: int64

In [54]:
smells.groupby('smell_identifier').count()

,project_name,file_path,start_line,start_column
smell_identifier,,,,
emptyValue,3,3,3,3
lookupUsage,5,5,5,5
useRelational,50,50,50,50


In [88]:
smells.groupby(['smell_identifier', 'project_name']).count()

file_path  \
smell_identifier project_name                                        
emptyValue       Requarks/wiki                                   1   
                 abcdabcd987/p2dv.in                             1   
                 impronunciable/hackdash                         1   
lookupUsage      EasyERP/EasyERP_open_source                     5   
useRelational    EasyERP/EasyERP_open_source                    44   
                 Lambda-School-Labs/CS10-restaurant-pos          1   
                 XimeraProject/server                            1   
                 codefordenver/Comrad                            3   
                 voluntarily/vly2                                1   

                                                         start_line  \
smell_identifier project_name                                         
emptyValue       Requarks/wiki                                    1   
                 abcdabcd987/p2dv.in                              1   
                 impronunciable/hackdash                          1   
lookupUsage      EasyERP/EasyERP_open_source                      5   
useRelational    EasyERP/EasyERP_open_source                     44   
                 Lambda-School-Labs/CS10-restaurant-pos           1   
                 XimeraProject/server                             1   
                 codefordenver/Comrad                             3   
                 voluntarily/vly2                                 1   

                                                         start_column  
smell_identifier project_name                                          
emptyValue       Requarks/wiki                                      1  
                 abcdabcd987/p2dv.in                                1  
                 impronunciable/hackdash                            1  
lookupUsage      EasyERP/EasyERP_open_source                        5  
useRelational    EasyERP/EasyERP_open_source                       44  
                 Lambda-School-Labs/CS10-restaurant-pos             1  
                 XimeraProject/server                               1  
                 codefordenver/Comrad                               3  
                 voluntarily/vly2                                   1

In [55]:
for i in smells_occ.index:
    projects_stats.at[i, 'smell_count'] = smells_occ[i]

In [56]:
for i in schemas.index:
    projects_stats.at[i, 'schemas_count'] = schemas.at[i, 'count(*)']

In [57]:
projects_stats.loc[projects_stats['dbaccesses'] > 0]


,stars,loc,dbaccesses,smell_count,schemas_count
repo,,,,,
0xTracker/0x-event-extractor,31.0,1344.0,7.0,NaN,2.0
17koa/koa-generator-examples,485.0,553.0,6.0,NaN,1.0
18F/checklistomania,362.0,1660.0,27.0,NaN,NaN
1956-studio/wechat-ship,13.0,2609.0,49.0,NaN,11.0
36node/sketch,17.0,8099.0,4.0,NaN,1.0
...,...,...,...,...,...
zaidka/genieacs,514.0,30.0,188.0,NaN,NaN
zipfworks/ember-prerender,141.0,1255.0,3.0,NaN,NaN
zishon89us/node-cheat,413.0,3233.0,19.0,NaN,3.0


In [58]:
projects_stats.to_csv('projects_stats.csv')

In [59]:
smells.groupby('smell_identifier').count()

,project_name,file_path,start_line,start_column
smell_identifier,,,,
emptyValue,3,3,3,3
lookupUsage,5,5,5,5
useRelational,50,50,50,50


In [60]:
smells.groupby('project_name').count()

,file_path,start_line,start_column,smell_identifier
project_name,,,,
EasyERP/EasyERP_open_source,49,49,49,49
Lambda-School-Labs/CS10-restaurant-pos,1,1,1,1
Requarks/wiki,1,1,1,1
XimeraProject/server,1,1,1,1
abcdabcd987/p2dv.in,1,1,1,1
codefordenver/Comrad,3,3,3,3
impronunciable/hackdash,1,1,1,1
voluntarily/vly2,1,1,1,1


,project_name,file_path,start_line,start_column,smell_identifier
0,codefordenver/Comrad,server/v1/controllers/playlists/recentPlays.js,107,25,useRelational
1,codefordenver/Comrad,server/v1/controllers/reporting/charting.js,39,3,useRelational
2,codefordenver/Comrad,server/v1/controllers/reporting/soundExchangeR...,30,3,useRelational
3,Requarks/wiki,server/setup.js,305,31,emptyValue
4,Lambda-School-Labs/CS10-restaurant-pos,api/controllers/employees/getAllServers.js,23,3,useRelational
5,impronunciable/hackdash,seo.js,85,5,emptyValue
6,EasyERP/EasyERP_open_source,handlers/opportunity.js,1953,13,lookupUsage
7,EasyERP/EasyERP_open_source,handlers/opportunity.js,2115,21,lookupUsage
8,EasyERP/EasyERP_open_source,handlers/opportunity.js,2233,13,lookupUsage
9,EasyERP/EasyERP_open_source,handlers/opportunity.js,2310,9,lookupUsage


In [91]:
for i in smells.index:
    smells.at[i, 'hash'] = commits.at[smells.at[i, 'project_name'], 'last_commit_hash']
smells.to_csv('evaluation_smells.csv')

In [61]:
leveraged =  projects_stats.loc[projects_stats['loc'] > 500]
leveraged = leveraged.loc[leveraged['stars'] >= 10]
leveraged

,stars,loc,dbaccesses,smell_count,schemas_count
repo,,,,,
0xTracker/0x-event-extractor,31.0,1344.0,7.0,NaN,2.0
17koa/koa-generator-examples,485.0,553.0,6.0,NaN,1.0
18F/checklistomania,362.0,1660.0,27.0,NaN,NaN
1956-studio/wechat-ship,13.0,2609.0,49.0,NaN,11.0
36node/sketch,17.0,8099.0,4.0,NaN,1.0
...,...,...,...,...,...
zackster/CompassionPit--Node-,21.0,98180.0,35.0,NaN,11.0
zipfworks/ember-prerender,141.0,1255.0,3.0,NaN,NaN
zishon89us/node-cheat,413.0,3233.0,19.0,NaN,3.0


In [62]:
for i in leveraged.index:
    leveraged.at[i, 'hash'] = commits.at[i, 'last_commit_hash']

In [63]:
leveraged.fillna(0).astype({'stars':'int', 'loc':'int', 'dbaccesses':'int', 'smell_count':'int', 'schemas_count':'int'}).to_csv('projects_stats.csv')
leveraged

,stars,loc,dbaccesses,smell_count,schemas_count,hash
repo,,,,,,
0xTracker/0x-event-extractor,31.0,1344.0,7.0,NaN,2.0,6b30a6453a857d7ca1661393a922005859c5ee8d
17koa/koa-generator-examples,485.0,553.0,6.0,NaN,1.0,59de28e6e46b11db8a1face1adc0cde41b83d248
18F/checklistomania,362.0,1660.0,27.0,NaN,NaN,98657747c5460c51debb6e396e4ade254e63df53
1956-studio/wechat-ship,13.0,2609.0,49.0,NaN,11.0,70a6ce289836c4d0878185a80e35b7e0758b11fa
36node/sketch,17.0,8099.0,4.0,NaN,1.0,4116403070b6f8a832c789dab6c213109d5c473d
...,...,...,...,...,...,...
zackster/CompassionPit--Node-,21.0,98180.0,35.0,NaN,11.0,dc3f023c2e8bb06e2155c27d02f7cabeac9dee66
zipfworks/ember-prerender,141.0,1255.0,3.0,NaN,NaN,d6eeb0df4524ac54f06b3cda0c755b7e66079bb3
zishon89us/node-cheat,413.0,3233.0,19.0,NaN,3.0,e73252e81aed263c9639a94b6f97506e8aa036a2


In [64]:
end_mongoose = mongoose_projects.intersection(leveraged.index)
len(end_mongoose)

802

In [65]:
end_mongodb = mongodb_projects.intersection(leveraged.index)
len(end_mongodb)

608

In [66]:
len(end_mongoose.intersection(end_mongodb))

298

In [67]:
leveraged['stars'].min()

10.0

In [68]:
leveraged['stars'].quantile(0.25)

20.75

In [69]:
leveraged['stars'].quantile(0.5)

54.5

In [70]:
leveraged['stars'].quantile(0.75)

213.5

In [71]:
leveraged['stars'].max()

357673.0

In [72]:
leveraged['loc'].min()

503.0

In [73]:
leveraged['loc'].quantile(0.25)

1817.5

In [74]:
leveraged['loc'].quantile(0.5)

5758.0

In [75]:
leveraged['loc'].quantile(0.75)

21105.75

In [76]:
leveraged['loc'].max()

850017.0

In [77]:
leveraged['dbaccesses'].min()

1.0

In [78]:
leveraged['dbaccesses'].quantile(0.25)

8.0

In [79]:
leveraged['dbaccesses'].quantile(0.5)

19.0

In [80]:
leveraged['dbaccesses'].quantile(0.75)

46.0

In [81]:
leveraged['dbaccesses'].max()

2196.0

In [82]:
leveraged['schemas_count'].min()

1.0

In [83]:
leveraged['schemas_count'].quantile(0.25)

2.0

In [84]:
leveraged['schemas_count'].quantile(0.5)

3.0

In [85]:
leveraged['schemas_count'].quantile(0.75)

7.0

In [86]:
leveraged['schemas_count'].max()

108.0